In [60]:
import exchange_calendars as xcals
from datetime import datetime
import pandas as pd

exchanges = xcals.get_calendar_names(include_aliases=False)
today = datetime.today().strftime('%Y-%m-%d')
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

today_calendars = pd.DataFrame()
for exchange in exchanges:
    cal = xcals.get_calendar(exchange)
    df = cal.schedule.loc[today:today]
    df['exchange'] = exchange
    df['is_open'] = cal.is_trading_minute(now)
    today_calendars = pd.concat([today_calendars, df], ignore_index=True)
    
continuous = today_calendars[today_calendars['break_start'].isnull()][['open','close','exchange','is_open']]
morning = today_calendars[~today_calendars['break_start'].isnull()][['open','break_start','exchange','is_open']].rename(columns={"break_start": "close"})
afternoon = today_calendars[~today_calendars['break_start'].isnull()][['break_end','close','exchange','is_open']].rename(columns={"break_end": "open"})

today_calendars = pd.concat([continuous, morning, afternoon], ignore_index=True)
today_calendars = today_calendars.sort_values(['open','close'])

exchanges_info = pd.read_csv('exchanges.csv')

today_calendars = today_calendars.merge(exchanges_info, how = 'left', left_on = 'exchange', right_on = 'ISO Code')

/tmp/ipykernel_2924055/4203674644.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2924055/4203674644.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2924055/4203674644.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ip

In [61]:
import plotly.express as px
fig = px.timeline(today_calendars, x_start="open", x_end="close", y="交易所",color="is_open")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up


fig.update_layout(shapes=[
    dict(
      type='line',
      yref='paper', y0=0, y1=1,
      xref='x', x0=now, x1=now
    )
])